<a href="https://colab.research.google.com/github/a-forty-two/diamler_17_jan/blob/main/09_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BODY -> body of string from which we will dervice our vectors
corpus_raw = 'Richie Rich is rich. The rich person is a happy person. He and She are not rich.'
corpus_lower = corpus_raw.lower()
corpus_lower

'richie rich is rich. the rich person is a happy person. he and she are not rich.'

In [4]:
# Segmentation-> break into sentences
# Tokenization-> getting every word separately
words = []
for word in corpus_lower.split():
  if word == '.':
    # if punctuation is not individual word but a part of word-> bye.
    word = word.replace('.', '') # every punctuation that we don't want should be filtered here
    words.append(word)
  else:
    words.append(word)

print(words)
# TO CONVERT TO VECTORS , we don't want repeating! 
# Which DS allows us to avoid repeating? SET!!
print(len(words))
unique_words = set(words)
print(unique_words)
print(len(unique_words))

['richie', 'rich', 'is', 'rich.', 'the', 'rich', 'person', 'is', 'a', 'happy', 'person.', 'he', 'and', 'she', 'are', 'not', 'rich.']
17
{'rich', 'the', 'is', 'not', 'and', 'are', 'person', 'happy', 'he', 'person.', 'rich.', 'she', 'richie', 'a'}
14


In [5]:
word2int = {} 
int2word = {}
vocab_size = len(unique_words)

# enumerate when passed a list, iterates on that list in index,value format!
# a-> [a,b,c]
# enumerate(a) -> (0,a), (1,b), (2,c)
for i,word in enumerate(unique_words):
  word2int[word] =i 
  int2word[i] = word

print(word2int) # Encoder Dict
print(int2word) # Decoder Dict
print(word2int['rich'])

{'rich': 0, 'the': 1, 'is': 2, 'not': 3, 'and': 4, 'are': 5, 'person': 6, 'happy': 7, 'he': 8, 'person.': 9, 'rich.': 10, 'she': 11, 'richie': 12, 'a': 13}
{0: 'rich', 1: 'the', 2: 'is', 3: 'not', 4: 'and', 5: 'are', 6: 'person', 7: 'happy', 8: 'he', 9: 'person.', 10: 'rich.', 11: 'she', 12: 'richie', 13: 'a'}
0


In [7]:
# SENTENCE level tokens so that we know what a complete sample looks like!
# please note: i have only 3 sentences, so dicitonary, prediction everything is going
# to be very poor!

raw_sentences = corpus_raw.split('.') # now breaking sentences instead of ' '
sentences = []
for sentence in raw_sentences:
  sentences.append(sentence.split()) # SPLIT -> BOTH SENTENCE and WORD tokens are created in 1go!

print(sentences) # contains extra [] because of the LAST '.' 
# sentences split the last full stop also became and element! 
sentences = sentences[:-1]
sentences

[['Richie', 'Rich', 'is', 'rich'], ['The', 'rich', 'person', 'is', 'a', 'happy', 'person'], ['He', 'and', 'She', 'are', 'not', 'rich'], []]


[['Richie', 'Rich', 'is', 'rich'],
 ['The', 'rich', 'person', 'is', 'a', 'happy', 'person'],
 ['He', 'and', 'She', 'are', 'not', 'rich']]

In [8]:
# i wanted to create word prediction
# ML algo?
# y-> next word, x -> previous word
# y = weights * x + bias 
# next_word = weights * previous_word + bias 

# BASICALLY what i need is ALL possible pairs of WORDs that can follow each other!!!

window_size = 2 # 2 words at a time
data = [] 
for sentence in sentences: # loop in sentence
  for wordId, word in enumerate(sentence): # loop in word
    for nxword in sentence[max(wordId-window_size,0):min(wordId+window_size,len(sentence))+1]:  
      if nxword!=word:
        data.append([word,nxword])
         # every word that could come next to it! 
      # no. of elements = len-window_size 
      # max -> 0, 1 at that point 0-2 and 1-2 will give me neg nos. hence error
      # to fix it we used max with 0! 
      # min -> at the end of array-> to avoid crossing the max len of array! 
      # +1 because the last element is excluded in numpy ranges [1,2]-> from 1 till 1!

print(data)


[['Richie', 'Rich'], ['Richie', 'is'], ['Rich', 'Richie'], ['Rich', 'is'], ['Rich', 'rich'], ['is', 'Richie'], ['is', 'Rich'], ['is', 'rich'], ['rich', 'Rich'], ['rich', 'is'], ['The', 'rich'], ['The', 'person'], ['rich', 'The'], ['rich', 'person'], ['rich', 'is'], ['person', 'The'], ['person', 'rich'], ['person', 'is'], ['person', 'a'], ['is', 'rich'], ['is', 'person'], ['is', 'a'], ['is', 'happy'], ['a', 'person'], ['a', 'is'], ['a', 'happy'], ['a', 'person'], ['happy', 'is'], ['happy', 'a'], ['happy', 'person'], ['person', 'a'], ['person', 'happy'], ['He', 'and'], ['He', 'She'], ['and', 'He'], ['and', 'She'], ['and', 'are'], ['She', 'He'], ['She', 'and'], ['She', 'are'], ['She', 'not'], ['are', 'and'], ['are', 'She'], ['are', 'not'], ['are', 'rich'], ['not', 'She'], ['not', 'are'], ['not', 'rich'], ['rich', 'are'], ['rich', 'not']]


In [ ]:
# Label Encoding: CONVERTING words to numbers is called LABEL encoding 
# [hello, word] -> [1,2 ]

# One-Hot Encoding: we converted categorical values into columns containing 1 and 0 only
# Sales_ City_             Sales_  City_BLR City_Mum 
#  100    BLR    ->         100       1       0
#  200    MUM               200       0       1

# Multi-Hot Encoding : A sentence is matched against a dictionary, and existing words are 
# marked hot (1), absent words are marked cold (0)
# Dictionary : { good, boy, am, I, not, green} 
# Sentence: green not good 
# Multi_hot_encoding:   1 0 0 0 1 1
# in multi hot encoding, words lose their positional meaning! 



In [9]:
import numpy as np
def Encoder(datapointindex, vocabsize):
  temp = np.zeros(vocabsize) # this is an ALL-COLD VECTOR! 
  temp[datapointindex] = 1 # wherever word exists, datapointindex is it's position
  return temp

trainx = []
trainy = []

In [10]:
for worddata in data: # worddata[0]-> prev word (train-input), [1]-> next word (train-label!)
  try:
    trainx.append(Encoder(word2int[worddata[0]], vocab_size))
    trainy.append(Encoder(word2int[worddata[1]], vocab_size))
  except:
    continue


# adjusting just for this demo
trainx = trainx[:len(trainy)]
trainx = np.asarray(trainx)
trainy = np.asarray(trainy)
print(trainx.shape)
print(trainy.shape)

(28, 14)
(28, 14)


In [11]:
trainx[:5]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
trainy[:5]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])